# 项目实战：RAG企业知识库（下）Part B

## 课程目标
通过本课程，学员将掌握RAG系统的Web前端开发、语音交互功能实现、系统功能扩展以及部署流程，构建一个完整的、用户友好的智能问答系统。

## 课程内容
1. Web前端开发与用户体验
2. 语音交互功能
3. 系统优化和功能拓展
4. 系统部署和运行

## 前提条件
- Python 3.8+ 和 FastAPI 后端环境
- 了解 HTML、CSS 和 JavaScript 基础
- 前端组件库：`vue`, `axios`, `marked`, `highlight.js`
- 熟悉 RAG 后端 API（如文档上传、流式问答）

## 1. Web前端开发与用户体验

### 1.1 Vue.js响应式开发
Vue.js活动的响应式前端框架，用于构建动态聊天界面，提供流畅的用户体验。

**关键点**：
- 使用 Vue 3 的 Composition API 或 Options API
- 实现组件化开发（如聊天窗口、侧边栏）
- 结合 Bootstrap 实现响应式布局

**代码示例**：Vue.js 聊天界面

In [ ]:
# 示例文件：static/chat.html
<!DOCTYPE html>
<html lang="zh-CN">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>RAG Demo - 对话</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
  <script src="https://cdn.jsdelivr.net/npm/vue@3.2.47/dist/vue.global.prod.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/axios@1.4.0/dist/axios.min.js"></script>
</head>
<body>
  <div id="app" class="container-fluid">
    <div class="chat-container" ref="chatContainer">
      <div v-for="(msg, index) in messages" :key="index" class="message" :class="{ user: msg.role === 'user', bot: msg.role === 'bot' }">
        <div class="message-content">{{ msg.content }}</div>
      </div>
    </div>
    <div class="input-container">
      <input v-model="userInput" @keyup.enter="sendMessage" type="text" class="form-control" placeholder="输入您的问题...">
      <button class="btn btn-primary" @click="sendMessage">发送</button>
    </div>
  </div>

  <script>
    const { createApp } = Vue;
    axios.defaults.baseURL = 'http://localhost:8000';

    createApp({
      data() {
        return {
          userInput: '',
          messages: []
        };
      },
      methods: {
        async sendMessage() {
          if (!this.userInput.trim()) return;
          this.messages.push({ role: 'user', content: this.userInput });
          this.userInput = '';
          try {
            const response = await axios.get(`/api/stream?query=${encodeURIComponent(this.messages[this.messages.length - 1].content)}`);
            this.messages.push({ role: 'bot', content: response.data.content });
          } catch (error) {
            this.messages.push({ role: 'bot', content: '错误: ' + error.message });
          }
        }
      }
    }).mount('#app');
  </script>
</body>
</html>

### 1.2 SSE客户端实现
Server-Sent Events (SSE) 用于接收后端的流式响应，实时展示 AI 回答。

**关键点**：
- 使用 `EventSource` API 连接 SSE 端点
- 处理流式数据，动态更新界面
- 确保错误处理和连接关闭

**代码示例**：SSE 客户端实现

In [ ]:
# 在 Vue 组件中添加 SSE 支持
methods: {
  async sendMessage() {
    if (!this.userInput.trim()) return;
    const userMessage = this.userInput;
    this.messages.push({ role: 'user', content: userMessage });
    this.userInput = '';
    this.messages.push({ role: 'bot', content: '正在思考...' });

    const botMessageIndex = this.messages.length - 1;
    const apiUrl = `/api/stream?query=${encodeURIComponent(userMessage)}`;
    const eventSource = new EventSource(apiUrl);

    eventSource.onmessage = (event) => {
      const data = JSON.parse(event.data);
      if (data.done) {
        eventSource.close();
        return;
      }
      if (data.content) {
        if (this.messages[botMessageIndex].content === '正在思考...') {
          this.messages[botMessageIndex].content = '';
        }
        this.messages[botMessageIndex].content += data.content;
        this.$nextTick(() => {
          this.$refs.chatContainer.scrollTop = this.$refs.chatContainer.scrollHeight;
        });
      }
    };

    eventSource.onerror = () => {
      eventSource.close();
      this.messages[botMessageIndex].content = '服务器连接错误，请稍后再试';
    };
  }
}

### 1.3 Markdown与代码高亮
使用 Marked.js 解析 Markdown 格式，Highlight.js 实现代码高亮，提升回答的可读性。

**关键点**：
- 配置 Marked.js 支持 GFM（GitHub Flavored Markdown）
- 集成 Highlight.js 支持多种编程语言
- 动态渲染 Markdown 内容

**代码示例**：Markdown 和代码高亮

In [ ]:
# 在 chat.html 中添加 Marked.js 和 Highlight.js
<script src="https://cdn.jsdelivr.net/npm/marked@5.0.2/marked.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/highlightjs@11.9.0/highlight.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/highlightjs@11.9.0/styles/monokai-sublime.min.css" rel="stylesheet">

<script>
  marked.setOptions({
    highlight: function(code, lang) {
      const language = hljs.getLanguage(lang) ? lang : 'plaintext';
      return hljs.highlight(code, { language }).value;
    },
    gfm: true,
    breaks: true
  });

  createApp({
    methods: {
      renderMarkdown(text) {
        if (!text) return '';
        return marked.parse(text);
      }
    },
    template: `<div v-html="renderMarkdown(message.content)"></div>`
  });
</script>

## 2. 语音交互功能

### 2.1 Web Speech API
Web Speech API 提供浏览器原生的语音识别功能，支持中文输入。

**关键点**：
- 使用 `SpeechRecognition` 或 `webkitSpeechRecognition`
- 设置语言为 `zh-CN`
- 处理识别结果和错误

**代码示例**：语音识别

In [ ]:
# 在 Vue 组件中实现语音识别
methods: {
  initSpeechRecognition() {
    const SpeechRecognition = window.SpeechRecognition || window.webkitSpeechRecognition;
    if (SpeechRecognition) {
      this.recognition = new SpeechRecognition();
      this.recognition.lang = 'zh-CN';
      this.recognition.interimResults = true;
      this.recognition.continuous = true;

      this.recognition.onresult = (event) => {
        const lastResultIndex = event.results.length - 1;
        this.userInput = event.results[lastResultIndex][0].transcript;
      };

      this.recognition.onerror = (event) => {
        console.error('语音识别错误:', event.error);
        this.isRecording = false;
        alert('语音识别失败，请检查麦克风或浏览器支持！');
      };

      this.recognition.onend = () => {
        if (this.isRecording) this.recognition.start();
      };
    } else {
      alert('您的浏览器不支持语音识别功能！');
    }
  },
  toggleRecording() {
    if (!this.recognition) return;
    if (this.isRecording) {
      this.recognition.stop();
      this.isRecording = false;
    } else {
      this.userInput = '';
      this.recognition.start();
      this.isRecording = true;
    }
  }
}

### 2.2 语音合成应用
使用 Web Speech API 的语音合成功能将 AI 回答转为语音输出。

**关键点**：
- 使用 `SpeechSynthesisUtterance` 创建语音对象
- 设置语言为 `zh-CN`
- 提供播放/停止控制

**代码示例**：语音合成

In [ ]:
# 在 Vue 组件中实现语音合成
data() {
  return {
    speechSynthesis: window.speechSynthesis,
    speakingIndex: null
  };
},
methods: {
  toggleSpeech(text, index) {
    if (this.speakingIndex === index) {
      this.speechSynthesis.cancel();
      this.speakingIndex = null;
    } else {
      this.speechSynthesis.cancel();
      const utterance = new SpeechSynthesisUtterance(text);
      utterance.lang = 'zh-CN';
      utterance.onend = () => {
        this.speakingIndex = null;
      };
      this.speechSynthesis.speak(utterance);
      this.speakingIndex = index;
    }
  }
}

### 2.3 用户体验优化
设计自然的语音交互流程，提供清晰的反馈机制。

**优化策略**：
- 显示录音状态（如麦克风图标高亮）
- 自动播放 AI 回答的语音（可选开关）
- 提供错误提示和重试机制

**代码示例**：语音交互 UI

In [ ]:
# 在 chat.html 中添加语音交互 UI
<div class="input-group mb-3">
  <input v-model="userInput" @keyup.enter="sendMessage" type="text" class="form-control" placeholder="输入您的问题或点击麦克风...">
  <button class="btn btn-outline-secondary mic-btn" :class="{ recording: isRecording }" @click="toggleRecording" title="语音输入">
    <i class="bi bi-mic-fill"></i>
  </button>
  <button class="btn btn-primary" @click="sendMessage">发送</button>
</div>

<style>
.mic-btn.recording {
  background-color: #dc3545 !important;
  color: white !important;
}
</style>

## 3. 系统优化和功能拓展

### 3.1 多文档格式支持
扩展文档解析功能，支持 PDF 和 DOCX 文件。

**关键点**：
- 使用 `PyPDF2` 或 `pdfplumber` 解析 PDF
- 使用 `python-docx` 解析 DOCX
- 统一文本提取逻辑

**代码示例**：PDF 和 DOCX 解析

In [ ]:
from fastapi import FastAPI, UploadFile, File, HTTPException
import PyPDF2
from docx import Document
import os

app = FastAPI()

@app.post("/api/upload")
async def upload_document(file: UploadFile = File(...)):
    content_text = ""
    temp_file = f"temp_{file.filename}"

    try:
        content = await file.read()
        with open(temp_file, "wb") as f:
            f.write(content)

        if file.filename.endswith(".txt"):
            try:
                with open(temp_file, "r", encoding="utf-8") as f:
                    content_text = f.read()
            except UnicodeDecodeError:
                with open(temp_file, "r", encoding="gbk", errors="ignore") as f:
                    content_text = f.read()
        elif file.filename.endswith(".pdf"):
            with open(temp_file, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                for page in reader.pages:
                    content_text += page.extract_text() or ""
        elif file.filename.endswith(".docx"):
            doc = Document(temp_file)
            content_text = "\n".join([para.text for para in doc.paragraphs])
        else:
            raise HTTPException(status_code=400, detail="不支持的文件格式")

        return {"filename": file.filename, "content": content_text[:100]}
    finally:
        if os.path.exists(temp_file):
            os.remove(temp_file)

# 安装依赖
# !pip install PyPDF2 python-docx

### 3.2 多大模型配置和切换
支持多个大语言模型（如 GLM-4-plus、ChatGPT），通过配置动态切换。

**关键点**：
- 定义模型配置文件
- 动态加载 API 配置
- 提供模型切换接口

**代码示例**：多模型支持

In [ ]:
from openai import OpenAI
from fastapi import FastAPI, HTTPException

app = FastAPI()

# 模型配置文件
models_config = {
    "glm-4-plus": {
        "api_key": "your-glm-api-key",
        "base_url": "https://open.bigmodel.cn/api/paas/v4/",
        "model_name": "glm-4-plus"
    },
    "chatgpt": {
        "api_key": "your-openai-api-key",
        "base_url": "https://api.openai.com/v1/",
        "model_name": "gpt-3.5-turbo"
    }
}

class ModelClient:
    def __init__(self, model_name):
        config = models_config.get(model_name)
        if not config:
            raise ValueError(f"模型 {model_name} 未配置")
        self.client = OpenAI(api_key=config["api_key"], base_url=config["base_url"])
        self.model_name = config["model_name"]

    def chat(self, messages):
        return self.client.chat.completions.create(
            model=self.model_name,
            messages=messages
        )

@app.post("/api/switch_model")
async def switch_model(model_name: str):
    if model_name not in models_config:
        raise HTTPException(status_code=400, detail="无效的模型名称")
    return {"message": f"已切换到模型 {model_name}"}

# 示例：调用模型
client = ModelClient("glm-4-plus")
response = client.chat([
    {"role": "user", "content": "Hello, world!"}
])
print(response.choices[0].message.content)

### 3.3 会话内容导出
实现会话内容的导出功能，支持保存为 Markdown 文件。

**关键点**：
- 从 SQLite 数据库读取会话数据
- 格式化输出为 Markdown
- 提供前端下载功能

**代码示例**：会话导出

In [ ]:
from fastapi import FastAPI, HTTPException
import sqlite3

app = FastAPI()

@app.get("/api/chat/export/{session_id}")
async def export_session(session_id: str):
    conn = sqlite3.connect('chat_history.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()

    try:
        cursor.execute("SELECT role, content FROM messages WHERE session_id = ? ORDER BY id", (session_id,))
        messages = [dict(row) for row in cursor.fetchall()]

        if not messages:
            raise HTTPException(status_code=404, detail="会话未找到")

        content = "# 会话导出\n\n"
        for msg in messages:
            role = '用户' if msg['role'] == 'user' else '助手'
            content += f"## {role}\n{msg['content']}\n\n"

        return {
            "content": content,
            "filename": f"session_{session_id}.md"
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        conn.close()

# 前端下载逻辑
<script>
  methods: {
    async exportSession(sessionId) {
      try {
        const response = await axios.get(`/api/chat/export/${sessionId}`);
        const blob = new Blob([response.data.content], { type: 'text/markdown' });
        const url = window.URL.createObjectURL(blob);
        const link = document.createElement('a');
        link.href = url;
        link.download = response.data.filename;
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
        window.URL.revokeObjectURL(url);
      } catch (error) {
        console.error('导出失败:', error);
        alert('导出会话失败！');
      }
    }
  }
</script>

## 4. 系统部署和运行

### 4.1 网站域名申请和配置
购买域名并配置 DNS 解析以访问 RAG 系统。

**步骤**：
1. 选择域名提供商（如阿里云、GoDaddy）
2. 注册域名（如 `mychatbot.com`）
3. 配置 DNS 解析，将域名指向服务器 IP
4. 设置 CNAME 或 A 记录，确保访问稳定

**示例配置**（阿里云 DNS 示例）：
- 记录类型：A
- 主机记录：`@` 或 `www`
- 记录值：服务器公网 IP（如 `123.45.67.89`）
- TTL：600 秒

### 4.2 服务器采购和环境配置
购买云服务器并配置运行环境。

**推荐配置**：
- 云服务商：阿里云、腾讯云、AWS
- 服务器规格：2核4GB内存，50GB SSD
- 操作系统：Ubuntu 20.04 LTS

**环境配置步骤**：
1. 安装 Python 3.8+ 和 pip
2. 安装 Node.js 和 npm
3. 安装 Nginx 作为反向代理
4. 配置防火墙，开放 80、443 端口

**代码示例**：Nginx 配置

In [ ]:
# /etc/nginx/sites-available/rag-demo
server {
    listen 80;
    server_name mychatbot.com;

    location / {
        proxy_pass http://localhost:8000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection "upgrade";
    }
}

# 启用配置
# sudo ln -s /etc/nginx/sites-available/rag-demo /etc/nginx/sites-enabled/
# sudo nginx -t && sudo systemctl reload nginx

### 4.3 项目程序代码部署
将代码上传到服务器并运行 FastAPI 应用。

**部署步骤**：
1. 使用 Git 克隆项目代码
2. 安装 Python 依赖：`pip install -r requirements.txt`
3. 运行 FastAPI：`uvicorn app:app --host 0.0.0.0 --port 8000`
4. 使用 `pm2` 或 `supervisor` 管理进程

**代码示例**：使用 pm2 运行

In [ ]:
# 安装 pm2
# npm install -g pm2

# 启动 FastAPI 应用
# pm2 start "uvicorn app:app --host 0.0.0.0 --port 8000" --name rag-demo

# 设置开机自启
# pm2 startup
# pm2 save

# 查看运行状态
# pm2 list

## 总结与实践

### 关键点回顾
- **前端开发**：使用 Vue.js 和 SSE 构建响应式聊天界面
- **语音交互**：通过 Web Speech API 实现语音输入和输出
- **功能扩展**：支持多文档格式、多模型切换和会话导出
- **部署**：配置域名、服务器和代码运行环境

### 实践任务
1. 开发一个包含语音交互的聊天界面
2. 实现 PDF 文档上传和内容解析功能
3. 部署 RAG 系统到云服务器并配置域名访问
